In [7]:
import pytesseract
import cv2

In [8]:
def ocr(image):
    custom_config = r'--oem 3 --psm 6 '
    return pytesseract.image_to_string(image, config=custom_config)

In [9]:
import gradio as gr

In [24]:
demo = gr.Blocks()

In [25]:
with demo:
    gr.Markdown("# OCR Demo")
    with gr.Tabs():
        with gr.TabItem("Examples"):
            with gr.Row():
                rad = gr.components.Radio(
                    ['Example 1', 'Example 2'], label='Select Video and wait till it appears!')
                img1 = gr.components.Image()
                web_bt1 = gr.Button('Submit')
        with gr.TabItem("Upload"):
            with gr.Row():
                img2 = gr.components.Image(source='upload')
            web_bt2 = gr.Button('Submit')

    def action(choice):
        if choice == 'Example 1':
            return 'test-images/receipt-1.jpeg'
        elif choice == 'Example 2':
            return 'test-images/receipt-2.jpeg'
    rad.change(action, rad, img1)
    op = gr.components.Textbox(label='Output')

    def fn(video):
        string = ocr(video)
        return string

    web_bt2.click(fn, inputs=[img2], outputs=op)
    web_bt1.click(fn, inputs=[img1], outputs=op)


In [27]:
try:
    # demo.queue()
    demo.launch(inline=False, server_name="0.0.0.0",
                server_port=8083, max_threads=100)
except Exception as e:
    print(e)
    pass


Running on local URL:  http://localhost:8083/

To create a public link, set `share=True` in `launch()`.


In [23]:
demo.close()

Closing server running on port: 8082
